## Callback Example and Holoviews

In [ ]:
import importlib
import os
import lsst.daf.persistence as dafPersist
import source_plots as sp
import numpy as np
import pandas as pd
import holoviews as hv
import datashader as ds
from holoviews.operation.datashader import datashade, shade, dynspread, rasterize
from holoviews.operation import decimate
from holoviews import streams

hv.extension('bokeh','matplotlib')

In [ ]:
butler = dafPersist.Butler('/project/shared/data/Twinkles_subset/output_data_v2/')

In [ ]:
keys = list(butler.getKeys('calexp'))
values = butler.queryMetadata('calexp', keys)

In [ ]:
data_ids = []
for value in values:
    temp = {}
    for k, v in zip(keys, value):
        temp[k]=v
    data_ids.append(temp)

In [ ]:
data_ids[0]

In [ ]:
def get_pandas_dataid(butler, data_id):
    """
    Produce a pandas dataframe with S/G data from a butler and dataid, probably something similar
    exists already
    """
    calexp = butler.get('calexp', dataId=data_id)
    calib = calexp.getCalib()
    src = butler.get('src', dataId=data_id)
    fluxes = src.getPsfFlux()
    good_indexes = np.logical_and(fluxes > 0.0, np.isfinite(fluxes))
    # Model fluxes also need to check for negative and bad values.
    model_fluxes = src.getModelFlux()
    good_model_indexes = np.logical_and(model_fluxes > 0.0, np.isfinite(model_fluxes))
    good_indexes = np.logical_and(good_indexes, good_model_indexes)
    psf_mag = calib.getMagnitude(fluxes[good_indexes])
    cm_mag = calib.getMagnitude(model_fluxes[good_indexes])
    delta_mag = psf_mag - cm_mag
    extendedness = src['base_ClassificationExtendedness_value'][good_indexes]
    df = pd.DataFrame({'delta_mag':delta_mag, 'psf_mag':psf_mag, 'class':extendedness, 'visit':data_id['visit'], 'filter':data_id['filter']})
    return df

In [ ]:
# Loop over all visits (slow)
df = get_pandas_dataid(butler, data_ids[0])
for  i in range(1,10):
    df = pd.concat([df,get_pandas_dataid(butler,data_ids[i])])

In [ ]:
print(' Total objects = {}'.format(len(df)))

In [ ]:
%%opts Layout [height=500 vspace=0.1 hspace=0.1]
%%opts Scatter [height=400 width=400 tools=['hover']](color='k' marker='o' size=2)
Points = hv.Scatter(df , 'psf_mag', 'delta_mag')
Points_shader = dynspread(datashade(Points))

In [ ]:
hv.Layout(Points +  Points_shader).cols(2)

In [ ]:
# from http://holoviews.org/user_guide/Custom_Interactivity.html

In [ ]:
# Grab pointer strem
pointer = streams.PointerXY()


In [ ]:
# Associate a map with the pointer

pointer_dmap = hv.DynamicMap(lambda x, y: hv.Points([(x, y)]), streams=[pointer])
print(pointer.source is pointer_dmap)



In [ ]:
# last click
pointer.contents